In [ ]:
#Only run in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/Taskmaster
#!pip install tensorflow --upgrade
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [5]:
import small_rubiks as rubiks
import numpy as np
from student import student
from classroom import classroom
from teacher import teacher
from small_rubiks_neural_networks import student_network
from utils import dotdict
from keras import regularizers

task = rubiks.rubiks_task
setup = rubiks.rubiks_setup
rng = np.random.default_rng(seed=0)

#my_student_network = student_network.load("models/trained.h5")
params = dotdict({
            "residual_weights_reg" : None,
            "residual_bias_reg" : None,
            "relu_leak" : 0.1,
            "residual_units" : 300,
            "learning_rate" : 0.001,
            "residual_layers" : 4
})
my_student_network = student_network.create(params)
student_template = student(task, 50,my_student_network)
t = teacher(setup, lambda n : 1+rng.poisson(lam=3,size=n))
c = classroom(task, setup, t, student_template, n_students=1, max_steps=10)
#c.run_training_batch(n_problems=1,epochs_per_episode=3)


In [6]:
for i in range(10):
    c.run_training_batch(n_problems=1,epochs_per_episode=3)
    #if i % 10 == 0:
    #    my_student_network.save("models/t.keras")

Finished step 0, 1 out of 1 remain open.
Finished step 1, 1 out of 1 remain open.
Finished step 2, 1 out of 1 remain open.
Finished step 3, 1 out of 1 remain open.
Finished step 4, 1 out of 1 remain open.
Finished step 5, 1 out of 1 remain open.
Finished step 6, 1 out of 1 remain open.


KeyboardInterrupt: 

In [5]:
from small_rubiks_neural_networks import student_network
import numpy as np
import small_rubiks as rubiks
from utils import dotdict
from keras import regularizers

params = dotdict({
    "residual_weights_reg" : regularizers.l2(0),
    "residual_bias_reg" : regularizers.l2(0),
    "relu_leak" : 0.1,
    "residual_units" : 100,
    "learning_rate" : 0.001,
    "residual_layers" : 4
})

my_student_network = student_network()
my_student_network.build_state_network(params)

n=4096
rng = np.random.default_rng(seed=0)
states = np.arange(24)[None,:].repeat(n,axis=0)

for k in range(15):
    for i in range(30):
        _, states = rubiks.task_action(states,rng.choice(a=12,size=n))

    actions = rng.choice(12,size=n)
    _, next_states = rubiks.task_action(states,actions)
    my_student_network.fit_state(states,actions,next_states,epochs = 10)

Epoch 1/10
120/120 [==============================] - 3s 6ms/step - loss: 0.6253 - val_loss: 0.4650
Epoch 2/10
120/120 [==============================] - 0s 4ms/step - loss: 0.4461 - val_loss: 0.4431
Epoch 3/10
120/120 [==============================] - 0s 4ms/step - loss: 0.4259 - val_loss: 0.4346
Epoch 4/10
120/120 [==============================] - 0s 3ms/step - loss: 0.4116 - val_loss: 0.4288
Epoch 5/10
120/120 [==============================] - 0s 4ms/step - loss: 0.4003 - val_loss: 0.4252
Epoch 6/10
120/120 [==============================] - 0s 4ms/step - loss: 0.3905 - val_loss: 0.4197
Epoch 7/10
120/120 [==============================] - 0s 4ms/step - loss: 0.3819 - val_loss: 0.4166
Epoch 8/10
120/120 [==============================] - 0s 4ms/step - loss: 0.3740 - val_loss: 0.4122
Epoch 9/10
120/120 [==============================] - 0s 4ms/step - loss: 0.3671 - val_loss: 0.4092
Epoch 10/10
120/120 [==============================] - 0s 3ms/step - loss: 0.3602 - val_loss: 0.4064

In [ ]:
rubiks.print_coloring(states[0])
for i in range(12):
    rubiks.print_coloring(rubiks.task_action(states[0],i))

In [4]:
states = np.arange(24)[None,:]
actions = np.array([0])
prediction = my_student_network.predict_state(states,actions)
prediction.reshape(1,24,6), prediction.reshape(1,24,6).argmax(axis=2), rubiks.start_coloring[rubiks.task_action(states,actions)[1][0]]

1/1 [==============================] - 0s 21ms/step


(array([[[9.9710751e-01, 6.0619249e-07, 1.4759846e-04, 8.7940595e-07,
          2.7570887e-03, 9.7917059e-08],
         [1.1799331e-05, 9.0194135e-06, 3.2481094e-04, 2.0508978e-06,
          3.2106001e-08, 9.9986571e-01],
         [1.8193028e-08, 1.8223140e-01, 2.0851733e-01, 7.3667913e-04,
          4.2290463e-05, 7.5026892e-06],
         [9.9984759e-01, 2.5260138e-07, 6.5540661e-07, 3.5026612e-06,
          1.4787694e-03, 1.9846500e-06],
         [5.9099410e-08, 2.3021027e-03, 5.8814701e-02, 2.1045190e-08,
          3.1885473e-04, 2.2258115e-01],
         [2.0254830e-04, 6.2786501e-07, 3.3112528e-07, 9.9999064e-01,
          3.4769183e-09, 2.0986581e-03],
         [1.0000000e+00, 2.5368703e-07, 2.5786532e-08, 2.0778268e-09,
          7.2886853e-04, 5.1691532e-05],
         [1.6136362e-07, 3.9164751e-04, 2.0268655e-07, 1.9103825e-06,
          9.9992043e-01, 5.9300392e-06],
         [5.3146548e-10, 1.0813341e-05, 1.0000000e+00, 1.7140443e-04,
          7.0714083e-09, 2.1267698e-07],
 